# 0. Setup

### Either: Baseline

In [ ]:
!git clone https://github.com/benjaminheltzel/baseline-dino-tracker.git

import os

source_file = "baseline-dino-tracker"
destination_file = "dino-tracker"

os.rename(source_file, destination_file)

print(f"Renamed {source_file} to {destination_file}")

### Or: Fused Pipeline

In [ ]:
!git clone https://github.com/benjaminheltzel/diffused-dino-tracker.git

import os

source_file = "diffused-dino-tracker"
destination_file = "dino-tracker"

os.rename(source_file, destination_file)

print(f"Renamed {source_file} to {destination_file}")

### Installation

Install conda

In [ ]:
%cd dino-tracker
!pip install -q condacolab
import condacolab
condacolab.install()

Setup env & install requirements

In [ ]:
%cd dino-tracker
!conda create -n dino-tracker python==3.9 -y
import condacolab
condacolab.check()
!activate dino-tracker
!pip install -r requirements.txt

Check if data accessible

In [ ]:
%cd dino-tracker
!ls dataset/horsejump/video

Utils for saving results

In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/drive')

def copy_folder(source_folder, destination_drive_folder):
    """
    Copies a folder from the Colab environment to Google Drive.

    Args:
        source_folder (str): Path to the source folder in the Colab environment.
        destination_drive_folder (str): Path to the destination folder in Google Drive.
    """
    try:
        shutil.copytree(source_folder, destination_drive_folder, dirs_exist_ok=True)
        print(f"Folder '{source_folder}' has been copied to '{destination_drive_folder}'.")
    except Exception as e:
        print(f"Error copying '{source_folder}' to '{destination_drive_folder}': {e}")

# Run full pipeline

1.   Preprocessing: Extract dino embeddings, etc
2.   Train on video
3.   Run inference
4.   Create visualizations
5.   (optional) Save to Google Drive as backup

In [ ]:
import os
os.environ['ID'] = '0'
id = os.environ.get('ID')

!PYTHONPATH=$(pwd):$PYTHONPATH python ./preprocessing/main_preprocessing.py \
    --config ./config/preprocessing.yaml \
    --data-path ./dataset/tapvid-davis/$ID/

!PYTHONPATH=$(pwd):$PYTHONPATH python ./train.py \
    --config ./config/train.yaml \
    --data-path ./dataset/tapvid-davis/$ID/

!PYTHONPATH=$(pwd):$PYTHONPATH python ./inference_grid.py \
    --config ./config/train.yaml \
    --data-path ./dataset/tapvid-davis/$ID/ \
    --use-segm-mask

!PYTHONPATH=$(pwd):$PYTHONPATH python visualization/visualize_rainbow.py \
    --data-path ./dataset/tapvid-davis/$ID/ \
    --plot-trails

copy_folder(f"./dataset/tapvid-davis/{id}", f"/content/drive/My Drive/difdino/{id}")

# Benchmark

Run eval prep

In [ ]:
!PYTHONPATH=$(pwd):$PYTHONPATH python ./preprocessing/save_dino_embed_video.py \
    --config ./config/preprocessing.yaml \
    --data-path ./dataset/tapvid-davis/$ID/

!PYTHONPATH=$(pwd):$PYTHONPATH python inference_benchmark.py \
    --config ./config/train.yaml \
    --data-path ./dataset/tapvid-davis/$ID/ \
    --benchmark-pickle-path ./tapvid/tapvid_davis_data_strided_32-fix.pkl \
    --video-id $ID

**Single Video**

In [ ]:
import os
id = os.environ.get('ID')

!PYTHONPATH=$(pwd):$PYTHONPATH python ./eval/eval_single_video.py --dataset-root-dir ./dataset/tapvid-davis --benchmark-pickle-path ./tapvid/tapvid_davis_data_strided_32.pkl --out-file ./$ID_metrics.csv --dataset-type tapvid --video-idx $ID

**All 30 Videos**

In [ ]:
!PYTHONPATH=$(pwd):$PYTHONPATH python ./eval/eval_benchmark.py \
    --dataset-root-dir ./dataset/tapvid-davis \
    --benchmark-pickle-path ./tapvid/tapvid_davis_data_strided_32.pkl \
    --out-file ./tapvid/comp_metrics_davis.csv \
    --dataset-type tapvid